# Estimación de Valoración y Recomendación de Juegos en Steam  

**Grupo 6**  
- **Rodrigo Juanes**  
- Noé López  
- Marc Velasco  
- Pablo Selma  
- Carlos Ribes

---

### Descripción del Proyecto  
Este proyecto tiene como objetivo:
1. **Obtener reseñas de Steam** mediante técnicas de web scrapping.
2. **Procesar las reseñas**  para el correcto entrenamiento de los siguientes modelos.
1. **Determinar si un usuario ha recomendado un juego** basándose en el análisis de su opinión.  
2. **Sugerir juegos relacionados** a partir de un texto escrito, empleando las etiquetas de género y características de los juegos.  
3. **Detectar ironía en las reseñas**, para mejorar la interpretación de las valoraciones.  

---

### Tecnologías y Recursos  
- **WS/API:** SteamDB, AppReviews

## Requirements

Librerías necesarias para trabajar con el notebook.

In [1]:
!pip install -r "requirements.txt"

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/7.1 MB ? eta -:--:--
   ------------- -------------------------- 2.4/7.1 MB 12.2 MB/s eta 0:00:01
   --------------------------- ------------ 5.0/7.1 MB 12.1 MB/s eta 0:00:01
   ---------------------------------------- 7.1/7.1 MB 11.3 MB/s eta 0:00:00
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32917 sha256=ebefc8a986d3c32c5b9d0eb82e05405d0d68c7e16d30575bc4ea638eeeabe495
  Stored in directory: c:\users\pablo\appdata\local\pip\cache\wheels\d0\1a\23\f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words

   -------------------- ------------------- 1/2 [pyspellchecker]
   -------------------- ------------------- 1/2 [pyspellchecker]
   ---------------------------------------- 2/2 [pyspellchecker]



  DEPRECATION: Building 'stop_words' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'stop_words'. Discussion can be found at https://github.com/pypa/pip/issues/6334


## Carga de librerías y funciones

In [2]:
# Librerías necesarias
import spacy
import numpy as np
import requests
import pandas as pd
import re
import time

# Funiones necesarias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from langdetect import detect

In [3]:
!python -m spacy download es_core_news_md

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     - -------------------------------------- 2.1/42.3 MB 11.8 MB/s eta 0:00:04
     ---- ----------------------------------- 4.7/42.3 MB 12.4 MB/s eta 0:00:04
     ------ --------------------------------- 7.3/42.3 MB 12.6 MB/s eta 0:00:03
     --------- ----------------------------- 10.2/42.3 MB 12.5 MB/s eta 0:00:03
     ----------- --------------------------- 12.8/42.3 MB 12.4 MB/s eta 0:00:03
     -------------- ------------------------ 15.5/42.3 MB 12.5 MB/s eta 0:00:03
     ---------------- ---------------------- 18.4/42.3 MB 12.4 MB/s eta 0:00:02
     ------------------- ------------------- 21.0/42.3 MB 12.6 MB/s eta 0:00:02
     --------------------- ----------------- 23.1/42.3 MB 12.2 MB/s eta 0:00:02
     ----------------------- --------------- 26.0/42.3 MB 12.3 MB/s eta 0:00:02
     -------------------------- ------------ 28.6/42.3 MB 12.2 MB/s eta 0:00:02
     ---------------------------- ---------- 31.

# Web scrapping

El objetivo es realizar el estudio sobre una gran variedad de reviews. La mejor manera de obtener esta gran cantidad de datos es mediante *web scrapping*. Dividiremos nuestro *web scrapping* en dos partes.

* **Extracción de reseñas**
* **Obtener una lista de juegos**

## Extracción de reseñas.

Empezaremos nuestro *web scrapping* estudiando como extraer reviews de un juego concreto. Como ejemplo, usaremos el videojuego Dota 2.

El link asociado a las primeras $n$ reseñas del juego de steam con $id = \text{value}$ en lenguaje español es:
- https://store.steampowered.com/appreviews/value?json=1&num_per_page=n&language=spanish

A partir de esta API podemos obtener fácilmente las reseñas de cualquier juego.

In [2]:
# URL del endpoint de reseñas
url = 'https://store.steampowered.com/appreviews/570?json=1&num_per_page=100&language=spanish'

# Realiza la petición
response = requests.get(url)
data = response.json() 

# Itera por las reseñas
for review in data['reviews']:
    print("Usuario:", review['author']['steamid'])
    print("Comentario:", review['review'])
    print("¿Recomienda?", review['voted_up'])
    print('-' * 40)

Usuario: 76561198079669556
Comentario: Para los que nunca lo han jugado y vinieron por recomendación o por el motivo que sea, no instalen esta m1erda de juego, por favor hagan caso a mi advertencia, tengo 10 años jugando esto, y dota ha arruinado mi vida, cuando creo que por fin dejé de jugarlo, algo o alguien o un pase de batalla, aparece de la nada y regresas por más, nunca es suficiente, nunca te llena, pierdes mas de lo que ganas y hay gente que merece algo peor que el infierno, este juego arruina la existencia misma, hay desesperación, agonía, desolación, peruanos, Techies, sufrimiento y un fuerte impulso por el su1c11dio, hermanos, por favor alejense de este juego y vivan sus vidas, enamorense, tengan hijos, viajen, coman, F0ll3n, vivan la vida, disfruten lo que nunca pude disfrutar, aun hay tiempo para ustedes, ya es tarde para mi.
¿Recomienda? False
----------------------------------------
Usuario: 76561198288933432
Comentario: Un juego con excelsas mecanicas, pero con la comun

De toda la información obtenida, tan solo nos interesa el texto de la reseña, y si el juego ha sido recomendado.

In [3]:
reviews = [review['review'] for review in data['reviews'] ]                # Obtener reviews a partir del diccionario
ratings  = [review['voted_up'] for review in data['reviews'] ]             # Obterner ratings asociados a las reviews anteriores

dataframe = pd.DataFrame({'reviews' : reviews , 'ratings' : ratings })     # Transformamos a formato pd.DataFrama para aprovechar sus múltiples utilidades
dataframe.head(10)

,reviews,ratings
0,Para los que nunca lo han jugado y vinieron po...,False
1,"Un juego con excelsas mecanicas, pero con la c...",False
2,"Es muy entretenido, si tienes horas libres pue...",True
3,"La verdad esque es muy buen juego, lo actualiz...",True
4,"después de 3 años siguiendo jugando, puedo dec...",True
5,"Sinceramente esta bien es un buen juego, pero ...",True
6,Continuacion del Dota clasico del Warcraft 3 p...,True
7,Buen juego pero la comunidad es un desastre va...,False
8,"bueno el dota 2 , es una moba que ace cambiar ...",True
9,oh.. solo hay que decir que una ves dentro j...,True


## Obtener una lista de juegos

Realizamos *Web Scraping* en la página de Steam, portal de venta de videojuegos, para obtener cuales son los juegos más jugados. De esta forma nos aseguramos que tenemos gran cantidad de comentarios en todos los juegos.

Para realizar el *Web Scraping* utilizamos Selenium ya que necesitamos interactuar con la página de Steam. Concretamente necesitamos scrollear para alcanzar el final de las página y así que cargue más juegos.

In [4]:
# Necesitamos selenium para hacer scroll en la página y cargar elementos

n = 9 # Número de scrolls
      # La página empieza con 50 juegos y carga 50 más por scroll

# Configurar Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Inicializar driver
driver = webdriver.Chrome(options=chrome_options)

# Cargar la página
driver.get("https://store.steampowered.com/search/?untags=9130&filter=topsellers&ndl=1")

# Realizar scrolls
for _ in range(n):
    # Scroll hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Esperar a que cargue contenido nuevo
    time.sleep(2)

# Obtener el HTML después de todos los scrolls
html = driver.page_source

# Cerrar el navegador
driver.quit()

Una vez descargado el html, lo tratamos con *BeautifulSoup* y extraemos los nombre de los juegos así como sus ID.

In [5]:
soup = BeautifulSoup(html, "html.parser")

tabla = soup.find_all("div", id="search_resultsRows")[0]

# Generamos una lista de diccionarios con la id y el título del juego.

# NOTA: Algunos de los "juegos más vendidos" no son juegos, como Steam Deck.
# Se puede filtrar más adelante, ya que en
# https://store.steampowered.com/api/appdetails?appids=1675200
# aparece como "type": "hardware", en vez de "type": "game" o "type": "dlc".
juegos = []

for game in tabla.find_all("a"):
  juegos.append({'id': game['data-ds-appid'], 'title': game.find("span", class_="title").text})

Comprovemos que hemos conseguido una lista adecuada.

In [6]:
juegos[:20]

[{'id': '578080', 'title': 'PUBG: BATTLEGROUNDS'},
 {'id': '730', 'title': 'Counter-Strike 2'},
 {'id': '2349820', 'title': "Hero's Land"},
 {'id': '3017860', 'title': 'DOOM: The Dark Ages'},
 {'id': '1675200', 'title': 'Steam Deck'},
 {'id': '1091500', 'title': 'Cyberpunk 2077'},
 {'id': '553850', 'title': 'HELLDIVERS™ 2'},
 {'id': '3164500', 'title': 'Schedule I'},
 {'id': '490110', 'title': 'The Precinct'},
 {'id': '3489700', 'title': 'Stellar Blade™'},
 {'id': '292030', 'title': 'The Witcher 3: Wild Hunt'},
 {'id': '1903340', 'title': 'Clair Obscur: Expedition 33'},
 {'id': '2292800', 'title': 'Blacksmith Master'},
 {'id': '2138330', 'title': 'Cyberpunk 2077: Phantom Liberty'},
 {'id': '2669320', 'title': 'EA SPORTS FC™ 25'},
 {'id': '252490', 'title': 'Rust'},
 {'id': '3241660', 'title': 'R.E.P.O.'},
 {'id': '1973530', 'title': 'Limbus Company'},
 {'id': '381210', 'title': 'Dead by Daylight'},
 {'id': '1086940', 'title': "Baldur's Gate 3"}]

## Extracción de reseñas

Una vez ya tenemos una lista de juegos con Ids, pasamos a la extracción de reseñas de cada juego.

In [7]:
all_reviews = []

for juego in juegos:
    game_id = juego['id']
    game_title = juego['title']
    url = f'https://store.steampowered.com/appreviews/{game_id}?json=1&num_per_page=100&language=spanish'
    try:
        response = requests.get(url)
        data = response.json()
        for review in data.get('reviews', []):
            all_reviews.append({
                'game_title': game_title,
                'game_id': game_id,
                'review': review['review'],
                'recommended': review['voted_up']
            })
    except Exception as e:
        print(f"Error loading reviews for {game_title} ({game_id}): {e}")

reviews_df = pd.DataFrame(all_reviews)

En el dataset obtenido tenemos:
* **game_title**: Título del juego.
* **game_id**: Identificador del juego.
* **review**: La review.
* **recommended**: Si se ha recomendado o no el juego.

In [8]:
reviews_df.head()

,game_title,game_id,review,recommended
0,PUBG: BATTLEGROUNDS,578080,Más de 1000 horas y todavía no he ganado una s...,True
1,PUBG: BATTLEGROUNDS,578080,"muy buen juego, hace falta optimización gráfic...",True
2,PUBG: BATTLEGROUNDS,578080,"lo intente volver a jugar, no me deja entrar, ...",False
3,PUBG: BATTLEGROUNDS,578080,"""El battle royale original que puso de moda mo...",True
4,PUBG: BATTLEGROUNDS,578080,PUBG fue uno de los primeros juegos tipo battl...,True


# Procesado del texto

Si bien ya hemos obtenido una gran cantidad de reviews, cerrando el capítulo de *web scrapping*, ahora debemos preparar el texto para poder trabajar con este.

## Detección de idioma

Tan solo queremos trabajar con reviews en español, así que debemos detectar el idioma en el que está escrito cada review.

A pesar de que disponemos de la función *detect* para detectar el idioma de cada review, primero tenemos que decidir qué hacer con todas las reviews sobre las cuales no podemos inferir un idioma.

Primero, eliminaremos todas las reseñas que no contengan ni una sola letra.

In [9]:
# Elimina filas donde 'review' no contiene ninguna letra
reviews_df = reviews_df[reviews_df['review'].apply(lambda x: bool(re.search(r'[a-zA-ZáéíóúÁÉÍÓÚñÑ]', str(x))))].reset_index(drop=True)

A continuación, debemos identificar un notable tipo de review, el *ASCII art*.

El *ASCII art* consiste en hacer dibujos usando distintos metacarácteres del alfabeto *ASCII*. El *ASCII art* está muy presente en estas reviews, pero no nos interesa trabajar con el, pues solo complicaría el entrenamiento de los modelos.

Para identificar dichas reviews, intentaremos detectar el idioma con *detect*. Esta función no puede detectar el idoma de el texto formado mayoritariamente por metacarácteres, sirviendo como un gran detector de *ASCII art*.

In [10]:
languages = []

for review in reviews_df['review']:
    try:
        lang = detect(review)
    except Exception as e:
        lang = "art"
    languages.append(lang)

reviews_df['language'] = languages

Comprovemos si hemos logrado capturar el arte del que hablabamos.

In [11]:
art_reviews = reviews_df[reviews_df['language'] == 'art'].reset_index(drop=True)
print(art_reviews['review'].iloc[0])

Masterpiece

Absolute cinema / cine absoluto 

⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣴⣤⣤⣄⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⣴⣿⣿⣿⣿⢿⣿⣿⡅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⣿⣿⣥⣶⡽⣷⣾⣦⣸⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣿⣿⣟⣡⣯⠛⣿⢉⣹⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢻⣿⣿⠋⣸⣾⠟⠹⠉⠹⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣷⠶⠏⠉⠁⠀⠀⠀⠀⠀⠨⣿⠶⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⠴⠒⠈⠉⠉⠁⠒⠲⣼⡏⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣝⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡜⠁⠀⠀⠀⠀⠀⠀⠀⢠⣟⠃⠀⢠⣤⠄⠀⠀⠀⠀⢀⠀⠀⠀⠀⠀⠛⢻⣽⠖⠒⠒⠒⠤⣀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠀⠀⣀⡀⠀⠀⢀⣀⣴⡿⠀⡄⠀⠀⠈⢷⡄⠀⠀⠀⢸⡆⠀⠀⠀⠀⠀⠀⢸⡄⠀⠀⠀⠀⠈⢣⡀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢿⣰⠊⠁⠀⠀⣠⢍⣴⠛⠀⢀⡇⠀⠀⠀⠘⣿⡄⠀⠀⠀⣧⠀⠀⠀⠀⠀⡜⢾⣷⣦⡄⠀⠀⢀⠀⢧⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠇⠤⠤⣒⡮⠾⠿⠉⠀⠀⣾⠃⠀⠀⠀⠀⣿⣷⠀⠀⠀⣿⡅⠀⠀⠀⠀⢳⠀⠹⢿⣄⠀⠀⠈⢣⡼⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⢀⣠⣤⣾⠶⠾⠋⢉⠁⣀⠀⢀⣠⡾⠁⠀⠀⠀⢀⣸⣿⣿⣇⠀⣠⣿⣷⡀⠀⡀⠀⠈⢧⠀⠈⠹⠷⠦⣄⡘⢳⠀⠀⠀⠀
⠀⠀⠀⠀⣠⣾⣟⣋⣁⣈⣤⣤⡴⠖⢛⢁⣠⣶⠋⠀⠀⠀⠀⠀⣴⣿⣿⣿⣿⠿⠟⡟⠿⣷⣴⣷⡀⠀⠘⢷⡀⠀⠀⠈⠻⠛⠛⣦⡄⠀⠀
⠀⠀⠀⣴⣿⣿⣿⣿⡿⣿⣿⣇⣀⣠⣴⣾⡉⠀⠀⠀⠀⢀⣴⣾⣿⣿⡿⠋⢁⣀⣜⡡⢄⣈⠛⠻⣷⣦⡀⠀⠉⠀⠀⠀⢀⣀⣯⣻⣿⣆⠀
⠀⠀⢠⡿⣻⢿⡉⠀⠉⢾⣍⣛⡻⣿⣿⣿⣿⣿⣶⣶⣿⣿⣿⣿⢟⣉⡴⠋⠱⡿⢋⢀⠲⣀⣉⠵⠚⠙⢿⣤⣤⣼⠟⢹⣿⣿⢫⡏⠻⣿⡄
⠀⠀⠀⣰⠃⠀⠹⣦⡀⠈⠀⠀⠙⠿⣿⣿⣿⣿⣿⣿⣿⣿⠿⡟⢿⣿⣿⡋⠹⡿⠟⠻⠻⡏⠀⠀⠀⠀⢸⣿⡘⠀⠀⠀⠙⢁⡞⡇⠀⢸⠇
⠀⠀⠀⡟⠀⠀⠀⠉⢽⣶⣤⣤⡤⠼⠶⠲⠤⠤⣄⣽⠿⠁⠀⠀⠂⠀⠈⠻⣶⣷⣴⣶⠾⠿⡖⠶⠶⢤⣈⣿⣿⣦⣀⠀⢠⣎⣴⠏⠀⠀⠀
⠀⠀⡼⠁⠀⠀⣀⣶⠖⠋⠉⠀⠀⠀⠀⠀⠀⠰⣿⠉⠀⠀⠀⠀⠀⠀⠀⠀⠘⢿⡏⠁⠀⠀⠀⠀⠈⡉⠉⢻⣏⠙⠿⢿⣿⡿⠋⠀⠀⠀⠀
⠀⡼⠁⠀⢠⡶⠛⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀

Ahora sí, podemos trabajar tan solo con las reseñas en castellano.

In [12]:
reseñas_df = reviews_df[reviews_df['language'] == 'es'].reset_index(drop=True)
reseñas_df.head(10)

,game_title,game_id,review,recommended,language
0,PUBG: BATTLEGROUNDS,578080,Más de 1000 horas y todavía no he ganado una s...,True,es
1,PUBG: BATTLEGROUNDS,578080,"muy buen juego, hace falta optimización gráfic...",True,es
2,PUBG: BATTLEGROUNDS,578080,"lo intente volver a jugar, no me deja entrar, ...",False,es
3,PUBG: BATTLEGROUNDS,578080,"""El battle royale original que puso de moda mo...",True,es
4,PUBG: BATTLEGROUNDS,578080,PUBG fue uno de los primeros juegos tipo battl...,True,es
5,PUBG: BATTLEGROUNDS,578080,"Ta bueno, perfecto para desequilibrarte mental...",True,es
6,PUBG: BATTLEGROUNDS,578080,"La fisica aplicada a los movimientos, los conc...",True,es
7,PUBG: BATTLEGROUNDS,578080,"es la mea tula de juego, jueguen nomas cabros ...",True,es
8,PUBG: BATTLEGROUNDS,578080,injugable. mecanicas de disparo imposibles si ...,False,es
9,PUBG: BATTLEGROUNDS,578080,juego de batalla muy realista y bastante emoci...,True,es


## Limpieza y normalización del texto

El siguiente paso, muy necesario en nuestro contexto, es la corrección ortográfica: las reseñas de Steam muchas veces contienen _slangs_, insultos y un tono informal, lo cual dificulta cualquier tipo de clasificación. En este caso, haremos uso de la librería **spellchecker**:

In [ ]:
import re, string
from unidecode import unidecode
from stop_words import get_stop_words

nlp=spacy.load('es_core_news_md')
stop_words = get_stop_words('spanish')

pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation))) #selecciona símbolos de puntuación

def clean_text(text):
    """Limpiamos las menciones y URL del texto. Luego convertimos en tokens
    eliminamos los tokens que son signos de puntuación, convertimos en
    minúsculas y quitamos signos de puntuación. Para terminar
    volvemos a convertir en cadena de texto"""
    text = re.sub(r'@[\w_]+|https?://[\w_./]+', '', text) #elimina menciones y URL
    tokens = nlp(text)
    tokens = [tok.lower_ for tok in tokens if not tok.is_punct and not tok.is_space]
    filtered_tokens = [pattern2.sub('', token) for token in tokens if not (token in stop_words)] #obvia stop_words y después quita signos de puntuación
    filtered_text = ' '.join(filtered_tokens)

    return filtered_text

def lemmatize_text(text):
    """Convertimos el texto a tokens, extraemos el lema de cada token
    y volvemos a convertir en cadena de texto sin acentos"""
    tokens = nlp(text)
    lemmatized_tokens = [tok.lemma_ for tok in tokens]
    lemmatized_text = ' '.join(lemmatized_tokens)
    deaccented_str = unidecode(lemmatized_text)

    return deaccented_str

In [30]:
# Añade columna con el texto limpio
reseñas_df['clean_text'] = reseñas_df['review'].apply(clean_text)

# Añade columna con la lematización del texto limpio
reseñas_df['lemmatized_text'] = reseñas_df['clean_text'].apply(lemmatize_text)

In [35]:
# Reordena las columnas para que 'clean_text' y 'lemmatized_text' estén después de 'review'
columnas = list(reseñas_df.columns)
columnas.remove('language')

# Reordena las columnas
nueva_orden = columnas.copy()
if 'review' in columnas and 'clean_text' in columnas and 'lemmatized_text' in columnas:
    idx = columnas.index('review')
    # Quita las columnas para reinsertarlas en orden correcto
    nueva_orden.remove('clean_text')
    nueva_orden.remove('lemmatized_text')
    nueva_orden = (
        nueva_orden[:idx+1] +
        ['clean_text', 'lemmatized_text'] +
        nueva_orden[idx+1:]
    )

reseñas_df = reseñas_df[nueva_orden]

In [36]:
reseñas_df.head(10) 

,game_title,game_id,review,clean_text,lemmatized_text,recommended
0,PUBG: BATTLEGROUNDS,578080,Más de 1000 horas y todavía no he ganado una s...,más 1000 horas y todavía no he ganado sola vez...,más 1000 hora y todavía no haber ganar solo ve...,True
1,PUBG: BATTLEGROUNDS,578080,"muy buen juego, hace falta optimización gráfic...",muy buen juego hace falta optimización gráfica...,mucho buen juego hacer falta optimización gráf...,True
2,PUBG: BATTLEGROUNDS,578080,"lo intente volver a jugar, no me deja entrar, ...",intente volver jugar no deja entrar tiene prob...,intentar volver jugar no dejar entrar tener pr...,False
3,PUBG: BATTLEGROUNDS,578080,"""El battle royale original que puso de moda mo...",battle royale original que puso moda morir sin...,battle royale original que poner moda morir si...,True
4,PUBG: BATTLEGROUNDS,578080,PUBG fue uno de los primeros juegos tipo battl...,pubg fue uno primeros juegos tipo battle royal...,pubg ser uno primero juegos tipo battle royale...,True
5,PUBG: BATTLEGROUNDS,578080,"Ta bueno, perfecto para desequilibrarte mental...",ta bueno perfecto para desequilibrarte mentalm...,to bueno perfecto para desequilibrarte mentalm...,True
6,PUBG: BATTLEGROUNDS,578080,"La fisica aplicada a los movimientos, los conc...",fisica aplicada movimientos conceptos sencillo...,fisico aplicado movimiento concepto sencillo q...,True
7,PUBG: BATTLEGROUNDS,578080,"es la mea tula de juego, jueguen nomas cabros ...",es mea tula juego jueguen nomas cabros es como...,ser mea tula juego jugar noma cabro ser como p...,True
8,PUBG: BATTLEGROUNDS,578080,injugable. mecanicas de disparo imposibles si ...,injugable mecanicas disparo imposibles si no e...,injugable mecanica disparo imposible si no ser...,False
9,PUBG: BATTLEGROUNDS,578080,juego de batalla muy realista y bastante emoci...,juego batalla muy realista y bastante emociona...,juego batalla mucho realista y bastante emocio...,True


## Exportación del dataset

Una vez preparado el datset, lo guardamos en formato .csv para el uso con modelos.

In [37]:
reseñas_df.to_csv('steam_reseñas.csv', index=False, encoding='utf-8')

## Extracción de la información de los juegos

Obtenemos mediante web scraping información en español de los juegos de los cuales hemos sacado los comentarios: un resumen del juego, los géneros y las categorias del juego.

In [26]:
ids = [game['id'] for game in juegos]
game_description_list = []
genres_list = []
categories_list = []

for id in ids:
  url = 'https://store.steampowered.com/api/appdetails?appids=' + id + '&l=spanish'

  # Realiza la petición
  response_2 = requests.get(url)
  data_2 = response_2.json()

  game_description_list.append(re.sub(r'<.+?>', ' ', data_2[id]['data']['detailed_description']))

  genres = []
  if 'genres' in data_2[id]['data'].keys():
    for genre in data_2[id]['data']['genres']:
      genres.append(genre['description'])
  genres_list.append(genres)

  categories = []
  if 'categories' in data_2[id]['data'].keys():
    for category in data_2[id]['data']['categories']:
      categories.append(category['description'])
  categories_list.append(categories)

  time.sleep(2)

Guardando en un dataset la información:
* **ids**: Identificador del juego.
* **game_description**: Resumen del juego.
* **genres**: Géneros del juego.
* **categories**: Categorías del juego.

In [27]:
game_description_df = pd.DataFrame({'game_id' : ids, 
                            'game_description' : game_description_list, 
                            'genres' : genres_list, 
                            'categories' : categories_list})
game_description_df.head(10)

,game_id,game_description,genres,categories
0,578080,"ATERRIZA, Aterriza en una selecc...","[Acción, Aventura, Multijugador masivo, Free t...","[Multijugador, JcJ, JcJ en línea, Estadísticas..."
1,730,"Durante las dos últimas décadas, Counter‑Strik...","[Acción, Free to Play]","[Multijugador, Multijugador multiplataforma, C..."
2,3017860,PREMIUM EDITION Resiste las acometidas y l...,[Acción],"[Un jugador, Logros de Steam, Compat. total co..."
3,1091500,Cyberpunk 2077: Ultimate Edition Descubr...,[Rol],"[Un jugador, Logros de Steam, Compat. total co..."
4,553850,Digital Deluxe Edition La edición incluye...,[Acción],"[Multijugador, Cooperativo, Cooperativo en lín..."
5,3164500,"Schedule I drugs, substances, or chemicals ar...","[Acción, Indie, Simuladores, Estrategia, Acces...","[Un jugador, Multijugador, Cooperativo, Cooper..."
6,490110,Discord Acerca del juego Eres el agente...,"[Acción, Indie, Simuladores]","[Un jugador, Logros de Steam, Compat. total co..."
7,292030,DESCUBRE LA HISTORIA PREVIA A THE WITCHER IV ...,[Rol],"[Un jugador, Logros de Steam, Cromos de Steam,..."
8,1903340,Digital Deluxe Edition La edición Deluxe i...,"[Acción, Rol]","[Un jugador, Logros de Steam, Compat. total co..."
9,2292800,En Blacksmith Master diriges una herrería med...,"[Simuladores, Estrategia, Acceso anticipado]","[Un jugador, Logros de Steam, Cromos de Steam,..."


## Exportación del dataset

Al igual que con las reseñas, guardamos el dataset en formato .csv para el uso con modelos.

In [28]:
game_description_df.to_csv('steam_descripciones_juegos.csv', index=False, encoding='utf-8')